In [ ]:
import numpy as np
import matplotlib.pyplot as plt

MODEL = "text-curie-001"

In [ ]:
# defining the function. At y=0 to get a 1D cut at the origin
def ackley_1d(x, offset=2.1):
    x = x - offset
    # the formula is rather large
    out = -(
        -20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2)))
        - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(0)))
        + np.e
        + 20
    )

    # returning
    return out


x = np.linspace(-5, 5, 200)
plt.plot(x, ackley_1d(x))

In [ ]:
import bolift
from langchain.prompts.prompt import PromptTemplate

## Few Shot

In [ ]:
asktell = bolift.AskTellFewShotMulti(
    x_formatter=lambda x: f"f({float(x):0.2f}) for the shifted Ackley function",
    model=MODEL,
    selector_k=10,
)

In [ ]:
N = 8
pool = bolift.Pool(list(x), formatter=asktell.format_x)
xi = pool.sample(1)[0]
# give one point
asktell.tell(xi, ackley_1d(xi))
point = []
pred_x = []
pred_y = []
true_y = []
for i in range(N):
    aq = "expected_improvement"
    if i == N - 1:
        aq = "greedy"
    px, _, py = asktell.ask(pool, k=4, aq_fxn=aq)
    xc = float(px[0])
    pool.choose(px[0])
    y = ackley_1d(xc)
    asktell.tell(xc, y)
    point.append((xc, y))
    pred_x.append([float(x) for x in px])
    pred_y.append(py)
    print(px[0], y)

In [ ]:
plt.plot(range(N), [y for x, y in point])

In [ ]:
def plot_one(i):
    plt.figure()
    plt.title(f"N={i}")
    plt.plot(x, ackley_1d(x), label="True")
    plt.scatter(
        [p[0] for p in point[:i]],
        [p[1] for p in point[:i]],
        color="C4",
        label="All selected",
    )
    plt.scatter(point[i][0], point[i][1], color="C3", label="Selected")
    sorted_idx = np.argsort(pred_x[i])
    plt.plot(
        np.array(pred_x[i])[sorted_idx],
        np.array(pred_y[i])[sorted_idx],
        label="Predicted",
    )
    plt.legend()
    plt.show()


plot_one(0)
plot_one(int(1 * N / 4))
plot_one(int(2 * N / 4))
plot_one(int(3 * N / 4))
plot_one(N - 1)

In [ ]:
print(asktell.prompt.format(x="0", i=10, y_name="y value"))

In [ ]:
asktell.predict(0), ackley_1d(0)

## Topk Few Shot

In [ ]:
asktell = bolift.AskTellFewShotTopk(
    x_formatter=lambda x: f"f({x}) for the shifted Ackley function",
    model="text-babbage-001",
)
asktell.predict(3)

In [ ]:
N = 10
pool = np.linspace(-5, 5, 50)
pool_str = [f"{x:0.2f}" for x in pool]
point = []
pred_x = []
pred_y = []
true_y = []
for i in range(N):
    aq = "expected_improvement"
    if i == N - 1:
        aq = "greedy"
    px, _, py = asktell.ask(pool_str, k=len(pool), aq_fxn=aq)
    xc = float(px[0])
    y = ackley_1d(xc)
    asktell.tell(xc, y)
    point.append((xc, y))
    pred_x.append([float(x) for x in px])
    pred_y.append(py)

In [ ]:
plt.plot(range(N), [y for x, y in point])

In [ ]:
def plot_one(i):
    plt.figure()
    plt.title(f"N={i}")
    plt.plot(pool, ackley_1d(pool), label="True")
    plt.scatter(point[i][0], point[i][1], color="C3", label="Selected")
    plt.scatter(
        [p[0] for p in point[:i]],
        [p[1] for p in point[:i]],
        color="C4",
        label="All selected",
    )
    sorted_idx = np.argsort(pred_x[i])
    plt.plot(
        np.array(pred_x[i])[sorted_idx],
        np.array(pred_y[i])[sorted_idx],
        label="Predicted",
    )
    plt.legend()
    plt.show()


plot_one(0)
plot_one(3)
plot_one(N - 1)

## AskTell Finetune

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import bolift
import os
import openai

def ackley_1d(x, offset=2.1):
    x = x - offset
    # the formula is rather large
    out = -(
        -20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2)))
        - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(0)))
        + np.e
        + 20
    )

    # returning
    return out


asktell = bolift.AskTellFinetuning(
    x_formatter=lambda x: f"f({x}) for the shifted Ackley function",
    model="text-babbage-001",
    # id="ft-NdChhQO5ojVByicsALqwHE9L",
    finetune=True,
)
print(f"f(3), {asktell.predict(3)}")
pool = np.linspace(-5, 5, 50)
pool_str = [f"{x:0.2f}" for x in pool]
point = []
pred_x = []
pred_y = []
true_y = []
aq = "greedy"
px, _, py = asktell.ask(pool_str, k=len(pool), aq_fxn=aq)
xc = float(px[0])
y = ackley_1d(xc)
asktell.tell(xc, y)
point.append((xc, y))
pred_x.append([float(x) for x in px])
pred_y.append(py)

print(f"f(3), {asktell.predict(3)}")

In [ ]:
N = 15
pool = np.linspace(-5, 5, 50)
pool_str = [f"{x:0.2f}" for x in pool]
point = []
pred_x = []
pred_y = []
true_y = []
for i in range(N):
    aq = "expected_improvement"
    if i == N - 1:
        aq = "greedy"
    px, _, py = asktell.ask(pool_str, k=len(pool), aq_fxn=aq)
    xc = float(px[0])
    y = ackley_1d(xc)
    asktell.tell(xc, y)
    point.append((xc, y))
    pred_x.append([float(x) for x in px])
    pred_y.append(py)

In [ ]:
asktell = bolift.AskTellFinetuning(
    x_formatter=lambda x: f"f({x}) for the shifted Ackley function",
    # id="ft-NdChhQO5ojVByicsALqwHE9L",
    model="text-babbage-001",
    finetune=False,
)

print(f"f(3), {asktell.predict(3)}")
print(ackley_1d(3))

In [ ]:
N = 11
pool = np.linspace(-5, 5, 100)
pool_str = [f"{x:0.2f}" for x in pool]
point = []
pred_x = []
pred_y = []
true_y = []
for i in range(N):
    aq = "expected_improvement"
    if i == N - 1:
        aq = "greedy"
    px, _, py = asktell.ask(pool_str, k=len(pool), aq_fxn=aq)
    xc = float(px[0])
    y = ackley_1d(xc)
    asktell.tell(xc, y)
    point.append((xc, y))
    pred_x.append([float(x) for x in px])
    pred_y.append(py)

In [ ]:
def plot_one(i):
    plt.figure()
    plt.title(f"N={i}")
    plt.plot(pool, ackley_1d(pool), label="True")
    plt.scatter(point[i][0], point[i][1], color="C3", label="Selected")
    plt.scatter(
        [p[0] for p in point[:i]],
        [p[1] for p in point[:i]],
        color="C4",
        label="All selected",
    )
    sorted_idx = np.argsort(pred_x[i])
    plt.plot(
        np.array(pred_x[i])[sorted_idx],
        np.array(pred_y[i])[sorted_idx],
        label="Predicted",
    )
    plt.legend()
    plt.show()


plot_one(0)
plot_one(5)
plot_one(N - 1)